In [1]:
!pip install -q datasets
!huggingface-cli login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 25.3 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `

In [4]:
from datasets import load_dataset
import re

# Load the dataset
dataset = load_dataset('deepanshdj/Osaat1_8k')

# Shuffle the dataset and slice it
dataset = dataset['train'].shuffle(seed=42).select(range(7831))

# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['chat']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Instruction:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Response:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}


# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)


Map:   0%|          | 0/7831 [00:00<?, ? examples/s]

In [5]:
transformed_dataset

Dataset({
    features: ['chat', 'text'],
    num_rows: 7831
})

In [10]:
transformed_dataset = transformed_dataset.remove_columns('chat')

In [13]:
transformed_dataset['text']

["<s>[INST] How would I make something similar to one of these simulation softwares? [/INST] To simulate a nuclear reactor in Python, we need to understand the basic principles of a nuclear reactor and the physics behind it.\n\nA nuclear reactor is a device that initiates and controls a sustained nuclear chain reaction. The heat produced in the reactor is used to generate electricity. The basic components of a nuclear reactor include fuel rods, control rods, coolant, and a moderator.\n\nTo write a program in Python for simulating a nuclear reactor, we can use the following steps:\n\n1. Define the initial conditions of the reactor, such as the temperature, pressure, coolant flow rate, and fuel type.\n2. Define the physics of the reactor, such as the neutron flux, fission rate, and heat generation rate.\n3. Define the control mechanisms of the reactor, such as the control rods and the coolant flow rate.\n4. Use numerical methods, such as finite difference or finite element methods, to so

In [14]:
transformed_dataset.push_to_hub("osaat1_8k_llama2")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/310 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/deepanshdj/osaat1_8k_llama2/commit/1fd4fb3230b358a7e4676bf09ce53456c115fb08', commit_message='Upload dataset', commit_description='', oid='1fd4fb3230b358a7e4676bf09ce53456c115fb08', pr_url=None, pr_revision=None, pr_num=None)